In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as matlab

In [2]:
dataset=pd.read_csv("preprocessed_data.csv")
dataset

,transaction_id,transaction_amount,location,merchant,age,gender,fraud_label
0,1,1000.0,New York,ABC Corp,35,M,0
1,2,500.0,Chicago,XYZ Inc,45,F,0
2,3,2000.0,Los Angeles,ABC Corp,28,M,1
3,4,1500.0,San Francisco,XYZ Inc,30,F,0
4,5,800.0,Chicago,ABC Corp,50,F,0
...,...,...,...,...,...,...,...
81,82,1500.0,Los Angeles,XYZ Inc,31,M,0
82,83,2800.0,San Francisco,ABC Corp,50,F,1
83,84,1350.0,Chicago,XYZ Inc,28,M,0
84,85,920.0,New York,ABC Corp,47,F,0


In [3]:
dataset=dataset.drop(['transaction_id'], axis= 1)

In [4]:
dataset

,transaction_amount,location,merchant,age,gender,fraud_label
0,1000.0,New York,ABC Corp,35,M,0
1,500.0,Chicago,XYZ Inc,45,F,0
2,2000.0,Los Angeles,ABC Corp,28,M,1
3,1500.0,San Francisco,XYZ Inc,30,F,0
4,800.0,Chicago,ABC Corp,50,F,0
...,...,...,...,...,...,...
81,1500.0,Los Angeles,XYZ Inc,31,M,0
82,2800.0,San Francisco,ABC Corp,50,F,1
83,1350.0,Chicago,XYZ Inc,28,M,0
84,920.0,New York,ABC Corp,47,F,0


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,transaction_amount,age,fraud_label,location_Los Angeles,location_New York,location_San Francisco,merchant_XYZ Inc,gender_M
0,1000.0,35,0,0,1,0,0,1
1,500.0,45,0,0,0,0,1,0
2,2000.0,28,1,1,0,0,0,1
3,1500.0,30,0,0,0,1,1,0
4,800.0,50,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
81,1500.0,31,0,1,0,0,1,1
82,2800.0,50,1,0,0,1,0,0
83,1350.0,28,0,0,0,0,1,1
84,920.0,47,0,0,1,0,0,0


In [6]:
dataset.columns

Index(['transaction_amount', 'age', 'fraud_label', 'location_Los Angeles',
       'location_New York', 'location_San Francisco', 'merchant_XYZ Inc',
       'gender_M'],
      dtype='object')

In [7]:
dataset["fraud_label"].value_counts()

0    70
1    16
Name: fraud_label, dtype: int64

In [8]:
independent=dataset.drop(['fraud_label'], axis= 1)

In [9]:
independent

,transaction_amount,age,location_Los Angeles,location_New York,location_San Francisco,merchant_XYZ Inc,gender_M
0,1000.0,35,0,1,0,0,1
1,500.0,45,0,0,0,1,0
2,2000.0,28,1,0,0,0,1
3,1500.0,30,0,0,1,1,0
4,800.0,50,0,0,0,0,0
...,...,...,...,...,...,...,...
81,1500.0,31,1,0,0,1,1
82,2800.0,50,0,0,1,0,0
83,1350.0,28,0,0,0,1,1
84,920.0,47,0,1,0,0,0


In [10]:
dependent=dataset[["fraud_label"]]
dependent

,fraud_label
0,0
1,0
2,1
3,0
4,0
...,...
81,0
82,1
83,0
84,0


In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [13]:
from sklearn.svm import SVC

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]} 



grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, Y_train)

In [ ]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test_) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)

In [ ]:
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)

In [ ]:
print(grid.best_params_)

In [ ]:
table=pd.DataFrame.from_dict(re)
table

In [ ]:
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

In [ ]:
print("The confusion Matrix:\n",cm)

In [ ]:
print("The report:\n",clf_report)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])